In [1]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.functions import col

db_path="s3a://tsel-bucket/data/warehouse/tablespace/managed/hive/"
spark = (SparkSession
    .builder
    .appName("driverapp-spark")
    .config("spark.sql.warehouse.dir", db_path)
    .config("spark.hadoop.fs.s2a.s3guard.ddb.region", "us-east-1")
    .config("spark.kerberos.access.hadoopFileSystem","s3a://tsel-bucket/")
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
    .master("local[5]") # should be possible to change this to SPARK on Yarn or SPARK on Kubernetes
    .getOrCreate())

In [2]:
# spark.sql("select event_timestamp, driver_id, conv_rate, acc_rate, avg_daily_trips, created from default.driver_stats").show()

In [3]:
parDF=spark.read.parquet("/home/cdsw/game_grouse/data/driver_hourly_stats.parquet")

In [4]:
parDF.show()

+-------------------+---------+-----------+-----------+---------------+--------------------+
|    event_timestamp|driver_id|  conv_rate|   acc_rate|avg_daily_trips|             created|
+-------------------+---------+-----------+-----------+---------------+--------------------+
|2022-11-14 07:00:00|     1003| 0.27977896|  0.7224932|             96|2022-11-29 07:44:...|
|2022-11-14 08:00:00|     1003|   0.809546| 0.73565924|            158|2022-11-29 07:44:...|
|2022-11-14 09:00:00|     1003| 0.09048953| 0.23551959|            516|2022-11-29 07:44:...|
|2022-11-14 10:00:00|     1003|0.050886042|   0.593193|            788|2022-11-29 07:44:...|
|2022-11-14 11:00:00|     1003|  0.4661726|  0.7025434|            941|2022-11-29 07:44:...|
|2022-11-14 12:00:00|     1003|  0.7483273| 0.99131227|            413|2022-11-29 07:44:...|
|2022-11-14 13:00:00|     1003| 0.08931428| 0.89642555|            957|2022-11-29 07:44:...|
|2022-11-14 14:00:00|     1003| 0.40904576| 0.24213125|            752

In [9]:
parDF.write \
    .mode("overwrite") \
    .format("parquet") \
    .saveAsTable("driver_stats1")

In [10]:
spark.sql("select * from driver_stats1").show()

+-------------------+---------+-----------+-----------+---------------+--------------------+
|    event_timestamp|driver_id|  conv_rate|   acc_rate|avg_daily_trips|             created|
+-------------------+---------+-----------+-----------+---------------+--------------------+
|2022-11-14 07:00:00|     1003| 0.27977896|  0.7224932|             96|2022-11-29 07:44:...|
|2022-11-14 08:00:00|     1003|   0.809546| 0.73565924|            158|2022-11-29 07:44:...|
|2022-11-14 09:00:00|     1003| 0.09048953| 0.23551959|            516|2022-11-29 07:44:...|
|2022-11-14 10:00:00|     1003|0.050886042|   0.593193|            788|2022-11-29 07:44:...|
|2022-11-14 11:00:00|     1003|  0.4661726|  0.7025434|            941|2022-11-29 07:44:...|
|2022-11-14 12:00:00|     1003|  0.7483273| 0.99131227|            413|2022-11-29 07:44:...|
|2022-11-14 13:00:00|     1003| 0.08931428| 0.89642555|            957|2022-11-29 07:44:...|
|2022-11-14 14:00:00|     1003| 0.40904576| 0.24213125|            752